In [1]:
## Install
!pip install easyocr

In [2]:
## Imports

import pandas as pd
import matplotlib.pyplot as plt
import time
from sklearn.metrics import roc_auc_score

import re
import string
import nltk
import itertools
import numpy as np
import os
import csv
import io

import torch
import cv2
import easyocr

%matplotlib inline
import warnings
warnings.filterwarnings("ignore")

# this allows you to apply a progress bar to pandas operations
from tqdm import tqdm
tqdm.pandas()

/home/studio-lab-user/.conda/envs/default/lib/python3.9/site-packages/scipy/__init__.py:146: UserWarning: A NumPy version >=1.16.5 and <1.23.0 is required for this version of SciPy (detected version 1.23.1
  warnings.warn(f"A NumPy version >={np_minversion} and <{np_maxversion}"


In [22]:
# easyocr loading
reader = easyocr.Reader(['en']) 

CUDA not available - defaulting to CPU. Note: This module is much faster with a GPU.


In [23]:
%cd /home/studio-lab-user/DS-Projects/OCR/OCR_images

/home/studio-lab-user/DS-Projects/OCR/OCR_images


In [24]:
# OCR_images에 있는 리스트들로 컬럼 하나 만들기. 리스트 이름 따기.
import pandas as pd
import os
image_df = pd.DataFrame(os.listdir(), columns=['image'])

In [25]:
image_df

,image
0,among (3).jpg
1,among (4).jpg
2,among (5).jpg
3,among (8).jpg
4,among (6).jpg
5,among (9).jpg
6,among (10).jpg
7,among (13).jpg
8,among (21).jpg
9,among (12).jpg


In [26]:
image_df
image_df.drop(index=107, inplace=True)

In [27]:
image_list = image_df['image'].tolist()

In [28]:
image_list

['among (3).jpg',
 'among (4).jpg',
 'among (5).jpg',
 'among (8).jpg',
 'among (6).jpg',
 'among (9).jpg',
 'among (10).jpg',
 'among (13).jpg',
 'among (21).jpg',
 'among (12).jpg',
 'among (15).jpg',
 'among (16).jpg',
 'among (17).jpg',
 'among (32).jpg',
 'among (18).jpg',
 'among (20).jpg',
 'among (11).jpg',
 'among (14).jpg',
 'among (22).jpg',
 'among (23).jpg',
 'among (25).jpg',
 'among (47).jpg',
 'among (28).jpg',
 'among (30).jpg',
 'among (59).jpg',
 'among (58).jpg',
 'among (24).jpg',
 'among (35).jpg',
 'among (26).jpg',
 'among (36).jpg',
 'among (41).jpg',
 'among (45).jpg',
 'among (29).jpg',
 'among (33).jpg',
 'among (64).jpg',
 'among (43).jpg',
 'among (65).jpg',
 'among (46).jpg',
 'among (66).jpg',
 'among (7).jpg',
 'among (38).jpg',
 'among (34).jpg',
 'among (52).jpg',
 'among (54).jpg',
 'among (50).jpg',
 'among (49).jpg',
 'among (57).jpg',
 'among (67).jpg',
 'among (56).jpg',
 'among (55).jpg',
 'among (51).jpg',
 'among (68).jpg',
 'among (42).jpg',


In [32]:
# words and scores
def get_words_and_scores_and_add_to_dataframe(row):
    photo = row["image"]
    if photo == "empty":
        return [[],[]]
    
    img_path = f"/home/studio-lab-user/DS-Projects/OCR/OCR_images/{photo}"
    # easyocr readtext
    img_text = reader.readtext(img_path)
    #empty lists
    words = []
    confidences = []
    #appending results into the lists
    for (bbox, text, prob) in img_text:
        words.append(text)
        confidences.append(prob)
    result = [words, confidences]
    return result

In [33]:
image_df["words"] = image_df.apply(get_words_and_scores_and_add_to_dataframe, axis=1)
image_df

,image,words
0,among (3).jpg,"[[(3, I.M P 0St 0.R, (2J], [0.1636072497092486..."
1,among (4).jpg,"[[REVER SIBLE!], [0.683081498916182]]"
2,among (5).jpg,"[[i93 -, 4646, 3, ssis, {, 1, 8], [0.012850255..."
3,among (8).jpg,"[[], []]"
4,among (6).jpg,"[[Goo0, Nonionu, SURF'SON, 6ed, MM, Rtch], [0...."
5,among (9).jpg,"[[], []]"
6,among (10).jpg,"[[], []]"
7,among (13).jpg,"[[Impostor, 1, 1, 3], [0.9998658453133549, 0.8..."
8,among (21).jpg,"[[], []]"
9,among (12).jpg,"[[Xri zijjrA $, chenging, ~Feee], [0.109949147..."


In [34]:
import re
import string
def clean_text(row):
    word_list = row["words"][0]
    new_word_list = []
    for word in word_list:
        new_word = word.lower()
        new_word = re.sub("[%s]" % re.escape(string.punctuation), " ", new_word)
        new_word_list.append(new_word)
    return [new_word_list, row["words"][1]]

In [35]:
image_df["words"] = image_df.apply(clean_text, axis=1)

In [36]:
## define the target words / phrases

targets = ["among us", "impostor"]

In [37]:
# helper functions for funal scoring function

def advance(iterator, step):
    next(itertools.islice(iterator, step, step), None)


def tuplewize(iterable, size):
    iterators = itertools.tee(iterable, size)
    for position, iterator in enumerate(iterators):
        advance(iterator, position)
    return zip(*iterators)

In [38]:
# define function to produce final scores

def get_final_score(row):
    sample_list = row["words"][0]
    score_list = row["words"][1]
    max_score = 0
    
    for target in targets:
        phrase_length = len(target.split())
        for c, s in zip(tuplewize(sample_list, phrase_length), tuplewize(score_list, phrase_length)):
            new_sample = " ".join(c)
            edit_distance = nltk.edit_distance(target, new_sample)
            closeness = (len(target)-edit_distance)/len(target)
            confidence_score = np.mean(s)
            final_score = closeness * confidence_score
            if final_score > max_score:
                max_score = final_score
             
    return max_score

In [39]:
import itertools
import nltk
import numpy as np

image_df["OCR_score"] = image_df.apply(get_final_score, axis=1)

In [40]:
image_df = image_df.sort_values(by='OCR_score', ascending=False)
pd.set_option('display.max_rows', None)
image_df

,image,words,OCR_score
126,among (138).jpg,"[[impostor, 6, 1, j, 3], [0.9999715928569747, ...",0.999972
13,among (32).jpg,"[[ mong us, impostor, mong us, impos, tor], [...",0.999940
14,among (18).jpg,"[[gbmong us, , impostor], [0.1642797729063458...",0.999938
7,among (13).jpg,"[[impostor, 1, 1, 3], [0.9998658453133549, 0.8...",0.999866
84,among (37).jpg,"[[impostor, 1, 1, 3, s], [0.9955174794211725, ...",0.995517
54,among (71).jpg,"[[impostor], [0.9944337324834492]]",0.994434
79,among (84).jpg,"[[1, mong, qmonc , qmong, mon u5 , 87, l on...",0.993758
147,among (1).jpg,"[[impostor, 3, , i, 1], [0.7982906357960781, ...",0.798291
83,among (88).jpg,"[[impostor, 1, i, 3, 6], [0.745935750393722, 0...",0.745936
68,among (74).jpg,"[[imposton], [0.797609109796984]]",0.697908


In [41]:
image_df.to_csv('/home/studio-lab-user/DS-Projects/OCR/easy_ocr.csv')